### Setting up a kernel

In [9]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as skaoai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

kernel = sk.Kernel()

useAzureOpenAI = True

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

deployment="gpt-4o"
service_id = "default"
kernel.add_service(
    skaoai.AzureChatCompletion(
        deployment_name=deployment,
        endpoint=endpoint,api_key=api_key)
    )

print("You made a kernel!")

You made a kernel!


### Loading the functions in the kernel

In [10]:
plugins_directory = "sk_plugins/native/v2"
car_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="car_plugins")
print(car_plugin)

name='car_plugins' description=None functions={'aircon': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='aircon', plugin_name='car_plugins', description='Operates the aircon of the car to adjust temperature', parameters=[KernelParameterMetadata(name='temperature', description='the new temperature', default_value=None, type_='float', is_required=True, type_object=<class 'float'>, schema_data={'type': 'number', 'description': 'the new temperature'})], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='the description of the operation that has been performed', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'the description of the operation that has been performed'}), additional_properties={}), method=<bound method CarAPI.aircon_temperature of <car_api.CarAPI object at 0x78dce59f3990>>, stream_method=None), 'GetCarStatus': KernelFunctionFromM

#### Using Autoinvoke, so semantic kernel decides when to call a plugin by itself

In [11]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior
from semantic_kernel.functions import KernelArguments

prompt = """
Consider the following car status {{car_plugins.GetCarStatus}}
Perform the following actions:
{{$input}}
"""

execution_settings = kernel.get_service(service_id).instantiate_prompt_execution_settings(
    service_id=service_id, 
    temperature=0,  # Temperature can affect the execution of plugins!
    max_tokens=2000,
    function_call_behavior=FunctionCallBehavior.AutoInvokeKernelFunctions(),
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The actions to be performed", is_required=True),
    ],        
    execution_settings=execution_settings,
 
)

car_function = kernel.add_function(
    function_name="car_function",
    plugin_name="CarFunctions",
    prompt_template_config=prompt_template_config,
)


In [12]:
summary = await kernel.invoke(car_function, input="Raise the passenger window")
print(summary)


Reading car status
Executing raise on window front_right
The passenger window (front right) has been raised.


In [13]:
summary = await kernel.invoke(car_function, input="Low temperature to 19 degrees")
print(summary)


Reading car status
Setting aircon to 19.0
Executing close on window rear_left
Executing close on window rear_right
The air conditioning has been set to 19 degrees, and the rear left and rear right windows have been closed.


In [14]:
summary = await kernel.invoke(car_function, input="I am feeling cold")

Reading car status
Setting aircon to 25.0
Executing close on window rear_left
Executing close on window rear_right


In [15]:
summary = await kernel.invoke(car_function, input="I am feeling hot")

Reading car status
Setting aircon to 18.0
Executing close on window rear_left
Executing close on window rear_right


In [17]:
summary = await kernel.invoke(car_function, input="The driver is cold, but the passenger is hot")

Reading car status
Setting aircon to 22.0
Executing close on window rear_left
Executing open on window front_right


In [16]:
summary = await kernel.invoke(car_function, input="Open the roof")

Reading car status
Invalid window name
Reading car status
